# Ensemble Result For CIFAR10
* 将各个阶段得到的网络进行集成学习，得到最终结果

**截止到2018年3月5日，融合的模型有：**
* Resnet200v2: bottleneck(erase first relu), add last bn.
* Resnet164v2: bottleneck(erase first relu), add last bn.
* Sparsenet100: growth_rate=36, bottleneck(erase_first_relu), add_last_bn.

In [1]:
import os

import mxnet as mx
import numpy as np
import pandas as pd

from mxnet import nd
from mxnet import gluon
from mxnet import image

from tqdm import tqdm

In [2]:
data_dir = '../data/kaggle_cifar10'
train_dir = 'train'
test_dir = 'test'
input_dir = 'train_valid_test'
label_file = 'trainLabels.csv'
valid_ratio = .1
input_str = data_dir + '/' + input_dir + '/'
model_dir = 'models'
model_result_dir = 'model_result'
batch_size = 128

In [34]:
def transform_train(data, label):
    im = data.astype('float32') / 255
    im = nd.transpose(im, (2,0,1))
    im = nd.expand_dims(im, axis=0)
    im = nd.pad(im, pad_width=(0,0,0,0,4,4,4,4), mode='constant', constant_value=0)[0]
    im = nd.transpose(im, (1,2,0))
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0,
                        rand_crop=True, rand_resize=False, rand_mirror=True,
                        mean=np.array([0.4914, 0.4822, 0.4465]),
                        std=np.array([0.2023, 0.1994, 0.2010]),
                        brightness=0, contrast=0,
                        saturation=0, hue=0,
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = data.astype('float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32),
                        mean=np.array([0.4914, 0.4822, 0.4465]),
                        std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

train_valid_ds = gluon.data.vision.ImageFolderDataset(input_str + 'train_valid', transform=transform_train)
test_ds = gluon.data.vision.ImageFolderDataset(input_str + 'test', transform=transform_test)
test_data = gluon.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, last_batch='keep')

In [15]:
def save_model_results(net, model_name, ctx=mx.gpu()):
    num_test = 300000
    net.load_params(os.path.join(data_dir, model_dir, model_name), ctx=ctx)
    outputs = nd.zeros(shape=(num_test, 10), ctx=ctx)
    for i, (data, _) in enumerate(tqdm(test_data)):
        output = net(data.as_in_context(ctx))
        outputs[i*batch_size:min((i+1)*batch_size, num_test), :] = output
        nd.waitall()
    nd.save(os.path.join(data_dir, model_result_dir, 'output_{}.nd'.format(model_name[:-7])), outputs)

In [16]:
def ensemble_model_result(preds, weight_list=None):
    if weight_list is None:
        weight_list = [1, ] * len(preds)
    output = nd.softmax(data=preds[0], axis=1) * weight_list[0]
    for i in range(1, len(preds)):
        output = output + nd.softmax(data=preds[i], axis=1) * weight_list[i]
    preds = nd.argmax(data=output, axis=1).astype(int).asnumpy() % 10
    return preds

In [17]:
model_list = ['cifar10-resnet200v2-bn-0.95490.params', 'cifar10-sparsenet_depth_100_growthrate_36_bn-0.95480.params', 
              'cifar10-resnet164v2-bn-0.95570.params']
model_name_list = [model_name[:-7] for model_name in model_list]
weight_list = [0.95490, 0.95480, 0.95570]

In [20]:
import modellib

sparsenet = modellib.SparseNet(num_classes=10, num_sparseblk_count=3, depth=100, growth_rate=36, 
                      bottleneck=True, verbose=True)
if not os.path.exists(os.path.join(data_dir, model_result_dir, 'output_{}.nd'.format(model_name_list[1]))):
    save_model_results(sparsenet, model_list[1])

100%|██████████████████████████████████████| 2344/2344 [13:28<00:00,  2.90it/s]


In [23]:
depth = 200
unit_list = [22, 22, 22]
filter_list = [16, 64, 128, 256]
resnet = modellib.ResNet(unit_list=unit_list, filter_list=filter_list, num_classes=10, 
                data_type='cifar10', bottle_neck=True, debug=True)
if not os.path.exists(os.path.join(data_dir, model_result_dir, 'output_{}.nd'.format(model_name_list[0]))):
    save_model_results(resnet, model_list[0])

100%|██████████████████████████████████████| 2344/2344 [12:44<00:00,  3.07it/s]


In [ ]:
depth = 164
unit_list = [18, 18, 18]
filter_list = [16, 64, 128, 256]
resnet = modellib.ResNet(unit_list=unit_list, filter_list=filter_list, num_classes=10, 
                data_type='cifar10', bottle_neck=True, debug=True)
if not os.path.exists(os.path.join(data_dir, model_result_dir, 'output_{}.nd'.format(model_name_list[2]))):
    save_model_results(resnet, model_list[2])

## Ensemble

In [35]:
preds = []
for model_result in os.listdir(os.path.join(data_dir, model_result_dir)):
    pred = nd.load(os.path.join(data_dir, model_result_dir, model_result))
    preds.append(pred[0])
    
final_prediction = ensemble_model_result(preds, weight_list)
sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': final_prediction})
df['label'] = df['label'].apply(lambda x: train_valid_ds.synsets[x])
df.to_csv('submission.csv', index=False)